In [1]:
USE_COLAB = True

In [2]:
import os
import sys

In [3]:
if USE_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')

    # go to the drive directory
    ROOT_PATH = '/content/drive/MyDrive/EPFL/Machine Learning/ml-project-2-odds-oddities'
    %cd $ROOT_PATH

else:
    ROOT_PATH = os.path.normpath(os.getcwd() + os.sep + os.pardir)

sys.path.append(ROOT_PATH)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/EPFL/Machine Learning/ml-project-2-odds-oddities


# Transformations

The aim of this notebook is to test different setups with transformations using cross-validation and measure top f1 score over all epochs and folds.

## Imports

In [4]:
import os
import cv2
import torch
import albumentations as A
import pandas as pd
import numpy as np
import segmentation_models_pytorch as smp

from sklearn.model_selection import KFold
from scripts.preprocessing import RoadDataset, split_data
from scripts.training import train_model
from torch.utils.data import DataLoader, SubsetRandomSampler

## Data

In [5]:
# specify train directory
train_directory = os.path.join(ROOT_PATH, 'data', 'raw', 'train')

In [6]:
# image paths so that all the images are used for train dataset (no test set for cv due to small training set)
image_path_train, _, mask_path_train, _ = split_data(train_directory, test_size=0)

# create train Dataset without transformations for now
train_dataset = RoadDataset(image_path_train, mask_path_train)

## Transformations

Define transformations we'll use in evaluating the performance of the model.

In [7]:
base_tf = [A.Resize(height=512, width=512, always_apply=True)]

tf_flip = [A.HorizontalFlip(p=0.5), A.VerticalFlip(p=0.5)]
tf_rotate = A.Rotate(p=0.5, limit=180, border_mode=cv2.BORDER_CONSTANT, rotate_method="ellipse")
tf_brightness = A.RandomBrightnessContrast(p=0.5)
tf_snow = A.RandomSnow(p=0.1)

added_tfs = np.array([tf_flip, tf_rotate, tf_brightness, tf_snow], dtype=object)

In [8]:
# initiate the setups for transformations
col_names = ['base', 'flip', 'rotate', 'brightness', 'snow']
masks = [
    [False, False, False, False],
    # [True, False, False, False],
    # [True, True, False, False],
    # [True, True, True, False],
    [True, True, True, True],
]

## Hyperparameters

Since our aim is to see, how different transformations influence the training, we fix the model, epochs and batch sizes.

In [9]:
ENCODER = 'resnet50'
ENCODER_WEIGHTS = 'imagenet'

SEED = 16
BATCH_SIZE = 4
K_FOLD = 3
N_CPU = os.cpu_count()
N_EPOCHS = 20

LOADER_PARAMS = {
    'batch_size': BATCH_SIZE,
    'num_workers': N_CPU,
    'persistent_workers': True
}

## Cross-Validation

In [10]:
metric_dict = {}

for setup in masks:

    print(f'setup: {str(setup)}')

    # get the picked tfs as list
    tfs = added_tfs[np.array(setup)]
    tf_selection = pd.Series(tfs).explode().tolist()

    k_fold = KFold(n_splits=K_FOLD, shuffle=True, random_state=SEED)

    # Record K-fold results in a (K_FOLD, num_epoch) matrix
    training_f1_matrix = []
    validation_f1_matrix = []

    train_tf = A.Compose(base_tf + tf_selection)
    valid_tf = A.Compose(base_tf)

    # Get training and validation indices
    for fold, (train_idx, val_idx) in enumerate(k_fold.split(train_dataset)):

        print(f'fold: {fold}')

        # Create training and validation loaders by providing current K-Fold train/validation indices to Sampler
        train_loader = DataLoader(train_dataset.set_tf(train_tf), sampler=SubsetRandomSampler(train_idx), **LOADER_PARAMS)
        valid_loader = DataLoader(train_dataset.set_tf(valid_tf), sampler=SubsetRandomSampler(val_idx), **LOADER_PARAMS)

        # Initialize model
        model_ = smp.create_model("FPN", encoder_name=ENCODER, encoder_weights=ENCODER_WEIGHTS)
        criterion_ = smp.losses.DiceLoss(smp.losses.BINARY_MODE, from_logits=True)
        optimizer_ = torch.optim.Adam(model_.parameters(), lr=0.0005)
        scheduler_ = torch.optim.lr_scheduler.CosineAnnealingLR(
            optimizer_,
            T_max=(len(train_loader.dataset) * N_EPOCHS) // train_loader.batch_size,
        )

        # Train model
        train_losses, valid_losses, train_f1s, valid_f1s = train_model(
            model_, (train_loader, valid_loader), criterion_, optimizer_, scheduler_, N_EPOCHS
        )

        # Save epoch results
        training_f1_matrix.append(np.array(train_f1s))
        validation_f1_matrix.append(np.array(valid_f1s))

    metric_dict[str(setup)] = validation_f1_matrix

setup: [False, False, False, False]
fold: 0


Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 126MB/s]
Epoch:   1. Train.      Loss: 0.547 | f1: 0.439: 100%|██████████| 17/17 [00:11<00:00,  1.44it/s]
Epoch:   1. Validation. Loss: 0.520 | f1: 0.455: 100%|██████████| 9/9 [00:01<00:00,  8.67it/s]
Epoch:   2. Train.      Loss: 0.338 | f1: 0.656: 100%|██████████| 17/17 [00:04<00:00,  3.83it/s]
Epoch:   2. Validation. Loss: 0.427 | f1: 0.553: 100%|██████████| 9/9 [00:00<00:00, 10.79it/s]
Epoch:   3. Train.      Loss: 0.279 | f1: 0.704: 100%|██████████| 17/17 [00:04<00:00,  3.87it/s]
Epoch:   3. Validation. Loss: 0.306 | f1: 0.657: 100%|██████████| 9/9 [00:00<00:00, 11.82it/s]
Epoch:   4. Train.      Loss: 0.219 | f1: 0.768: 100%|██████████| 17/17 [00:04<00:00,  3.95it/s]
Epoch:   4. Validation. Loss: 0.244 | f1: 0.720: 100%|██████████| 9/9 [00:00<00:00, 11.78it/s]
Epoch:   5. Train.      Loss: 0.184 | f1: 0.806

fold: 1


Epoch:   1. Train.      Loss: 0.538 | f1: 0.450: 100%|██████████| 17/17 [00:04<00:00,  3.46it/s]
Epoch:   1. Validation. Loss: 0.468 | f1: 0.518: 100%|██████████| 9/9 [00:00<00:00,  9.15it/s]
Epoch:   2. Train.      Loss: 0.341 | f1: 0.650: 100%|██████████| 17/17 [00:04<00:00,  3.52it/s]
Epoch:   2. Validation. Loss: 0.329 | f1: 0.654: 100%|██████████| 9/9 [00:00<00:00, 10.53it/s]
Epoch:   3. Train.      Loss: 0.273 | f1: 0.714: 100%|██████████| 17/17 [00:04<00:00,  3.45it/s]
Epoch:   3. Validation. Loss: 0.317 | f1: 0.666: 100%|██████████| 9/9 [00:00<00:00, 11.04it/s]
Epoch:   4. Train.      Loss: 0.214 | f1: 0.768: 100%|██████████| 17/17 [00:04<00:00,  3.54it/s]
Epoch:   4. Validation. Loss: 0.220 | f1: 0.747: 100%|██████████| 9/9 [00:00<00:00, 11.44it/s]
Epoch:   5. Train.      Loss: 0.190 | f1: 0.798: 100%|██████████| 17/17 [00:04<00:00,  3.55it/s]
Epoch:   5. Validation. Loss: 0.226 | f1: 0.740: 100%|██████████| 9/9 [00:00<00:00, 11.31it/s]
Epoch:   6. Train.      Loss: 0.152 | f1

fold: 2


...

Epoch:   1. Train.      Loss: 0.556 | f1: 0.432: 100%|██████████| 17/17 [00:05<00:00,  3.26it/s]
Epoch:   1. Validation. Loss: 0.417 | f1: 0.572: 100%|██████████| 9/9 [00:01<00:00,  7.44it/s]
Epoch:   2. Train.      Loss: 0.406 | f1: 0.580: 100%|██████████| 17/17 [00:05<00:00,  3.26it/s]
Epoch:   2. Validation. Loss: 0.396 | f1: 0.599: 100%|██████████| 9/9 [00:01<00:00,  7.69it/s]
Epoch:   3. Train.      Loss: 0.367 | f1: 0.624: 100%|██████████| 17/17 [00:05<00:00,  3.25it/s]
Epoch:   3. Validation. Loss: 0.331 | f1: 0.646: 100%|██████████| 9/9 [00:00<00:00, 10.44it/s]
Epoch:   4. Train.      Loss: 0.332 | f1: 0.657: 100%|██████████| 17/17 [00:04<00:00,  3.53it/s]
Epoch:   4. Validation. Loss: 0.299 | f1: 0.676: 100%|██████████| 9/9 [00:00<00:00, 10.84it/s]
Epoch:   5. Train.      Loss: 0.307 | f1: 0.682: 100%|██████████| 17/17 [00:04<00:00,  3.57it/s]
Epoch:   5. Validation. Loss: 0.284 | f1: 0.686: 100%|██████████| 9/9 [00:00<00:00, 11.06it/s]
Epoch:   6. Train.      Loss: 0.262 | f1

fold: 1


Epoch:   1. Train.      Loss: 0.586 | f1: 0.404: 100%|██████████| 17/17 [00:05<00:00,  3.38it/s]
Epoch:   1. Validation. Loss: 0.538 | f1: 0.456: 100%|██████████| 9/9 [00:00<00:00,  9.43it/s]
Epoch:   2. Train.      Loss: 0.402 | f1: 0.590: 100%|██████████| 17/17 [00:04<00:00,  3.54it/s]
Epoch:   2. Validation. Loss: 0.462 | f1: 0.536: 100%|██████████| 9/9 [00:00<00:00, 10.35it/s]
Epoch:   3. Train.      Loss: 0.348 | f1: 0.632: 100%|██████████| 17/17 [00:04<00:00,  3.41it/s]
Epoch:   3. Validation. Loss: 0.344 | f1: 0.640: 100%|██████████| 9/9 [00:00<00:00, 10.53it/s]
Epoch:   4. Train.      Loss: 0.329 | f1: 0.656: 100%|██████████| 17/17 [00:04<00:00,  3.51it/s]
Epoch:   4. Validation. Loss: 0.284 | f1: 0.695: 100%|██████████| 9/9 [00:00<00:00, 11.35it/s]
Epoch:   5. Train.      Loss: 0.272 | f1: 0.698: 100%|██████████| 17/17 [00:04<00:00,  3.54it/s]
Epoch:   5. Validation. Loss: 0.287 | f1: 0.696: 100%|██████████| 9/9 [00:00<00:00, 11.41it/s]
Epoch:   6. Train.      Loss: 0.261 | f1

fold: 2


Epoch:   1. Train.      Loss: 0.521 | f1: 0.468: 100%|██████████| 17/17 [00:05<00:00,  3.33it/s]
Epoch:   1. Validation. Loss: 0.460 | f1: 0.535: 100%|██████████| 9/9 [00:00<00:00,  9.48it/s]
Epoch:   2. Train.      Loss: 0.386 | f1: 0.600: 100%|██████████| 17/17 [00:04<00:00,  3.53it/s]
Epoch:   2. Validation. Loss: 0.359 | f1: 0.626: 100%|██████████| 9/9 [00:00<00:00, 10.81it/s]
Epoch:   3. Train.      Loss: 0.323 | f1: 0.655: 100%|██████████| 17/17 [00:04<00:00,  3.47it/s]
Epoch:   3. Validation. Loss: 0.301 | f1: 0.696: 100%|██████████| 9/9 [00:00<00:00,  9.97it/s]
Epoch:   4. Train.      Loss: 0.321 | f1: 0.656: 100%|██████████| 17/17 [00:04<00:00,  3.47it/s]
Epoch:   4. Validation. Loss: 0.306 | f1: 0.693: 100%|██████████| 9/9 [00:00<00:00, 11.35it/s]
Epoch:   5. Train.      Loss: 0.268 | f1: 0.702: 100%|██████████| 17/17 [00:04<00:00,  3.53it/s]
Epoch:   5. Validation. Loss: 0.261 | f1: 0.731: 100%|██████████| 9/9 [00:00<00:00, 11.30it/s]
Epoch:   6. Train.      Loss: 0.260 | f1

## Metrics

In [11]:
f1s = []
std_devs = []

for matrix in metric_dict.values():
    matrix = np.array(matrix)
    mean_per_epoch = matrix.mean(axis=0)

    best_epoch = mean_per_epoch.argmax()
    best_f1 = mean_per_epoch.max()

    f1s.append(mean_per_epoch.max())
    std_devs.append(matrix[:, best_epoch].std())

In [12]:
final_df = pd.DataFrame(index=metric_dict.keys(), columns=['top-f1', 'std'], data=np.array([f1s, std_devs]).T)

In [13]:
final_df

,top-f1,std
"[False, False, False, False]",0.815348,0.018222
"[True, True, True, True]",0.808037,0.016056


In [1]:
import matplotlib.pyplot as plt

arr = np.array(list(metric_dict.values())[0])


plt.fill_between(np.arange(arr.shape[1]), arr.mean(axis=0) - arr.std(axis=0), arr.mean(axis=0) + arr.std(axis=0), alpha = 0.5, color = 'gray')
plt.plot(arr.mean(axis=0), color = 'black')

arr = np.array(list(metric_dict.values())[1])


plt.fill_between(np.arange(arr.shape[1]), arr.mean(axis=0) - arr.std(axis=0), arr.mean(axis=0) + arr.std(axis=0), alpha = 0.5, color = 'green')
plt.plot(arr.mean(axis=0), color = 'green')

NameError: ignored